### What this notebook does

This notebook (PMT-profiles-fast.ipynb) loads and processes meteorological measurement data collected from a mast, a radiometer, and a sonic instrument for a specific date.

It automatically:

- Reads all .dat files from your local folders for the selected date
- Converts and cleans timestamped measurements

- Calculates: Air temperature at multiple heights (2–10 m), Wind speed, pressure, humidity, and air density and other thermodynamic and other quantities from the instruments. 

- Creates time-series plots for temperature, wind speed, air density, and related variables

- Merges all data into a single 10-minute averaged CSV file for further analysis

You only need to edit:

- date_str  = 'YYYY-MM-DD'     # the date to process
- data_folder = f"C:\\path\\to\\your\\data\\mast\\{month_str}\\{date_str}"
- data_folder_sonic = f"C:\\path\\to\\your\\radiometer\\{month_str}\\{date_str}"
- data_dir = f"C:\\path\\to\\your\\Sonic\\{month_str}\\{date_str}"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime

#!pip install pvlib
from matplotlib.dates import DateFormatter

from datetime import time
import pvlib
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.dates as mdates
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# Constants
Cp = 1005  # Specific heat capacity of dry air at constant pressure (J/kg/K)
g = 9.81   # Acceleration due to gravity (m/s^2)

A=6.11*100 #Pa
beta=0.067 #K^-1
Ttrip=273.16 #K
epsilon=0.622
sigma=5.67e-8 #W*m^-2*K^-4
Lv = 2.5e6  # Latent heat of vaporization in J/kg

Rd=287.04
Rv=461.5

rho_atm=1.225 #kg/m^3
m_co2=0.044 #kg/mole molecular mass CO2
m_atm=0.028 #molecular mass atmosphere

In [ ]:
# Function to calculate vapor pressure (e) from relative humidity (RH) and temperature (T)
def calculate_vapor_pressure(RH, T):
    #es = A*np.exp(beta*(T+273.15-Ttrip))  # Saturation vapor pressure in Pa
    es=610.78*np.exp(17.2694*(T+273.15-Ttrip)/(T+273.15-35.86))
    e = RH * es / 100  # Vapor pressure in Pa
    return e

# Function to calculate specific humidity (qv) from vapor pressure (e) and atmospheric pressure (p)
def calculate_specific_humidity(e, p):
    #qv = epsilon * e *1000 / (p*100) # Specific humidity in g/kg
    qv=e*1000/(p*100+(((Rv/Rd)-1)*(p*100-e)))
    return qv

#def calculate_saturation_specific_humidity(e, p):
 #   qs = epsilon * (e*100/RH) *1000 / (p*100) # Specific humidity in g/kg
  #  return qs

def calculate_saturation_specific_humidity(T, p):
    es=610.78*np.exp(17.2694*(T+273.15-Ttrip)/(T+273.15-35.86))
    qs=es*1000/(((Rv/Rd)*(p*100-es))+es)
    return qs



In [ ]:

# ⚠️ Edit this path to match your local setup before running.

date_str  = '2024-05-23'      # ← only line you normally edit
month_str = date_str[:7]      # '2024-05'

# Example placeholder path (safe to show publicly)
data_folder = f"C:\\path\\to\\your\\data\\mast\\{month_str}\\{date_str}"

# Get a list of all .dat files in the folder
data_files = [file for file in os.listdir(data_folder) if file.endswith('.dat')]

# Heights corresponding to each temperature measurement (in meters)
heights = [2,2.99,4.47,6.69,10]


# Initialize an empty DataFrame to store the combined data
all_data = []

# Initialize an empty list to store the dry static energy data for each height
all_dse_data = []
all_virtual_dse_data = []  # Initialize a list for virtual dry static energy


# Initialize an empty list to store the wind speed data for each height
all_ws_data = []

# Initialize an empty list to store the pressure data for height 2.99
all_pressure_data = [] #mbar=hPa

# Initialize an empty list to store the specific humidity data for each height
all_qv_data = []

all_qs_data=[]

# Initialize an empty list to store the relative humidity data for each height
all_rh_data = []

all_winddr_data=[]

all_windsdv_data=[]

na_counts = 0  # Initialize a counter for NaT values

# Loop through each file in the folder
for file in data_files:
    # Construct the full file path
    file_path = os.path.join(data_folder, file)
    
    # Read the data from the file
    data = pd.read_csv(file_path, skiprows=1)
    
    # Convert TIMESTAMP column to datetime format with error handling
    data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'], errors='coerce')
    
    # Count NaT values
    na_counts += data['TIMESTAMP'].isna().sum()
    # Drop rows with NaT values in the TIMESTAMP column
    data.dropna(subset=['TIMESTAMP'], inplace=True)
    
    # Select relevant columns containing temperature data
    temperature_columns = ['AirTC_E5567_Avg', 'AirTC_E5568_Avg', 'AirTC_E5569_Avg', 'AirTC_E5570_Avg','AirTC_E5571_Avg']  # Adjust column names if needed
    
    # Loop through each column and convert it to numeric type
    for column in temperature_columns:
        data[column] = pd.to_numeric(data[column], errors='coerce')
    # Append the data to the list
    all_data.append(data)
    # Print the number of NaT values
    print(f"Number of NaT values in TIMESTAMP column: {na_counts}")
    
    
    
    # Calculate dry static energy for each height
    for column, height in zip(temperature_columns, heights):
        # Calculate temperature in Kelvin
        data[f'Temperature_K_{height}'] = data[column] + 273.15
        
        # Calculate dry static energy
        data[f'Dry_Static_Energy_{height}'] = data[f'Temperature_K_{height}'] + g * height/Cp
    
    # Append the dry static energy data to the list
    all_dse_data.append(data)
    
    
    
    # Plot time series of temperature measurements for each file
    plt.figure(figsize=(10, 6))
    for column in temperature_columns:
        plt.plot(data['TIMESTAMP'], data[column], label=column)
    
    # Format the x-axis to show time in HH:MM format
    date_format = DateFormatter('%H:%M')
    plt.gca().xaxis.set_major_formatter(date_format)

    plt.xlabel('Time (UTC)')
    plt.ylabel('Temperature (°C)')
    plt.title(f'Time Series of Temperature Measurements - {file}')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig(os.path.join(data_folder, f'{file}_temperature_plot.jpg'))
    plt.close()
    
    # Select relevant columns containing wind speed data
    ws_columns = ['WS_ms_D15008_Avg', 'WS_ms_D15014_Avg', 'WS_ms_D15463_Avg']
    
    # Loop through each column and convert it to numeric type
    for column1 in ws_columns:
        data[column1] = pd.to_numeric(data[column1], errors='coerce')
    # Append the data to the list
    all_ws_data.append(data)
    
    
    # Plot time series of wind speed measurements for each file
    plt.figure(figsize=(10, 6))
    for column1 in ws_columns:
        plt.plot(data['TIMESTAMP'], data[column1], label=column1)
    
    # Format the x-axis to show time in HH:MM format
    date_format = DateFormatter('%H:%M')
    plt.gca().xaxis.set_major_formatter(date_format)

    plt.xlabel('Time (UTC)')
    plt.ylabel('Wind speed (m/s)')
    plt.title(f'Time Series of Wind Speed Measurements - {file}')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig(os.path.join(data_folder, f'{file}_windspeed_plot.jpg'))
    plt.close()
    
    
    # Select relevant columns containing pressyre data
    p_column = 'BP_mbar_Avg'
    
    # Loop through each column and convert it to numeric type
    data[p_column] = pd.to_numeric(data[p_column], errors='coerce')
    # Append the data to the list
    all_pressure_data.append(data)
    
    #select RH columns
    rh_columns = ['RH_E5567_Avg', 'RH_E5568_Avg', 'RH_E5569_Avg', 'RH_E5570_Avg', 'RH_E5571_Avg']
    
    # Loop through each column and convert it to numeric type
    for column in rh_columns:
        data[column] = pd.to_numeric(data[column], errors='coerce')
    
     
    # Create empty columns for specific humidity corresponding to each height
    for height in heights:
        data[f'qv_{height}m'] = np.nan
    
    # Calculate and plot specific humidity for each height
    for temp_column, rh_column, height in zip(temperature_columns, rh_columns, heights):
        # Calculate vapor pressure (e) using the temperature for each hour
        T = data[temp_column]  # Temperature in degrees Celsius
        RH = data[rh_column]   # Relative humidity as percentage
        
        e = calculate_vapor_pressure(RH, T)
        
        # Atmospheric pressure (p) at height 2.99m
        p = data[p_column]
        
        # Calculate specific humidity (qv) in g/kg
        qv = calculate_specific_humidity(e, p)
        
        qs= calculate_saturation_specific_humidity(T,p)
        
        
        # Assign specific humidity data to the corresponding column
        data[f'qv_{height}m'] = qv
        
        data[f'qs_{height}m'] = qs

        
        # Calculate virtual dry static energy
        data[f'Virtual_Dry_Static_Energy_{height}'] = data[f'Dry_Static_Energy_{height}'] + data[f'Temperature_K_{height}']*(((Rv / Rd) - 1) * (data[f'qv_{height}m']/1000))  # Convert g/kg to kg/kg by dividing by 1000
        
    # Append the virtual DSE data for the current height
    all_virtual_dse_data.append(data[['TIMESTAMP'] + [f'Virtual_Dry_Static_Energy_{height}' for height in heights]])
    # Append the specific humidity data to the list
    all_qv_data.append(data[['TIMESTAMP'] + [f'qv_{height}m' for height in heights]])
    
    all_qs_data.append(data[['TIMESTAMP'] + [f'qs_{height}m' for height in heights]])

    # Plot specific humidity for each height
    plt.figure(figsize=(10, 6))
    for height in heights:
        plt.plot(data['TIMESTAMP'], data[f'qv_{height}m'], label=f'Specific Humidity - Height: {height}m')
    plt.xlabel('Time (UTC)')
    plt.ylabel('Specific Humidity (g/kg)')
    plt.title(f'Specific Humidity (qv) - {file}')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # Save the plot
    plot_name = f'{file.split(".")[0]}_specific_humidity_plot.jpg'
    plt.savefig(os.path.join(data_folder, plot_name))
    plt.close()
    
    
    
    wind_dir_columns= ['WindDir_D15008_Avg','WindDir_D15014_Avg','WindDir_D15463_Avg']
    
    wind_dirsdv_columns =['WindDir_D15008_StDev','WindDir_D15014_StDev', 'WindDir_D15463_StDev']
    
    for column in wind_dir_columns:
        data[column] = pd.to_numeric(data[column], errors='coerce')
    # Append the data to the list
    all_winddr_data.append(data)
    
    for column in wind_dirsdv_columns:
        data[column] = pd.to_numeric(data[column], errors='coerce')
    # Append the data to the list
    
    all_windsdv_data.append(data)
    
    # Loop through each column and convert it to numeric type
    for column in rh_columns:
        data[column] = pd.to_numeric(data[column], errors='coerce')
    all_rh_data.append(data)
   
    
# Concatenate all data into a single DataFrame
all_dse_data = pd.concat(all_dse_data)

all_ws_data= pd.concat(all_ws_data)

all_pressure_data= pd.concat(all_pressure_data)

all_winddr_data=pd.concat(all_winddr_data)

all_windsdv_data=pd.concat(all_windsdv_data)

all_qv_data = pd.concat(all_qv_data, ignore_index=True)

# Concatenate all virtual DSE data into a single DataFrame
all_virtual_dse_data_df = pd.concat(all_virtual_dse_data, ignore_index=True)

all_rh_data= pd.concat(all_rh_data,ignore_index=True)
all_qs_data= pd.concat(all_qs_data,ignore_index=True)

combined_data = pd.concat(all_data)
print(all_qv_data)

In [ ]:
print(all_virtual_dse_data_df)

In [ ]:

# Plot time series of temperature measurements for the entire day
plt.figure(figsize=(10, 6))
for column in temperature_columns:
    plt.plot(combined_data['TIMESTAMP'], combined_data[column], label=column)
    
    
# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(date_format)

plt.xlabel('Time (UTC)')
plt.ylabel('Temperature (°C)')
plt.title('Time Series of Temperature Measurements for the Whole Day')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Save the plot as a JPEG file in the same folder
#output_file = os.path.join(data_folder, 'temperature_time_series.jpg')
#plt.savefig(output_file, format='jpg')
# Save the plot as a PDF file in the same folder
output_file = os.path.join(data_folder, 'temperature_time_series.pdf')
plt.savefig(output_file, format='pdf', dpi=300)  # Save as PDF for better quality
# Show the plot (optional)
plt.show()

# Plot the dry static energy over the whole day for each height
plt.figure(figsize=(10, 6))
for height in heights:
    plt.plot(all_dse_data['TIMESTAMP'], all_dse_data[f'Dry_Static_Energy_{height}'], label=f'{height}m')

    
# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(date_format)

plt.xlabel('Time (UTC)')
plt.ylabel('Dry Static Energy (over heat capacity) (K)')
plt.title('Dry Static Energy Variation Over the Whole Day')
plt.legend(title='Height', loc='upper right')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Save the plot as a JPEG file in the same folder
#output_file = os.path.join(data_folder, 'drystaticenergy_time_series.jpg')
#plt.savefig(output_file, format='jpg')

# Save the plot as a PDF file in the same folder
output_file = os.path.join(data_folder, 'drystaticenergy_time_series.pdf')
plt.savefig(output_file, format='pdf', dpi=300)  # Save as PDF for better quality
# Show the plot (optional)
plt.show()

# Plot the virtual dry static energy over the whole day for each height
plt.figure(figsize=(10, 6))
for height in heights:
    plt.plot(all_virtual_dse_data_df['TIMESTAMP'], all_virtual_dse_data_df[f'Virtual_Dry_Static_Energy_{height}'], label=f'{height}m')

# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(date_format)

plt.xlabel('Time (UTC)')
plt.ylabel('Virtual Dry Static Energy (over heat capacity) (K)')
plt.title('Virtual Dry Static Energy Variation Over the Whole Day')
plt.legend(title='Height', loc='upper right')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Save the plot as a PNG file in the same folder
output_file = os.path.join(data_folder, 'virtual_dry_static_energy_time_series.png')
plt.savefig(output_file, format='png', dpi=300)  # Save as PNG for better quality

# Show the plot (optional)
plt.show()



# Plot time series of temperature measurements for the entire day
plt.figure(figsize=(10, 6))
for column in ws_columns:
    plt.plot(all_ws_data['TIMESTAMP'], all_ws_data[column], label=column)
    
    
    
# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(date_format)

#plt.ylim(0, 15)  

plt.xlabel('Time')
plt.ylabel('Wind Speed (m/s)')
plt.title('Time Series of Wind Speed Measurements for the Whole Day')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Save the plot as a JPEG file in the same folder
#output_file = os.path.join(data_folder, 'windspeed_time_series.jpg')
#plt.savefig(output_file, format='jpg')

output_file = os.path.join(data_folder, 'windspeed_time_series.pdf')
plt.savefig(output_file, format='pdf', dpi=300)  # Save as PDF for better quality

# Show the plot (optional)
plt.show()

# Plot time series of temperature measurements for the entire day
plt.figure(figsize=(10, 6))

plt.plot(all_pressure_data['TIMESTAMP'], all_pressure_data[p_column], label=p_column)
    
    
# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(date_format)

plt.xlabel('Time')
plt.ylabel('Pressure at 2.99 m(mbar)')
plt.title('Time Series of Pressure Measurements for the Whole Day')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Save the plot as a JPEG file in the same folder
#output_file = os.path.join(data_folder, 'pressure_time_series.jpg')
#plt.savefig(output_file, format='jpg')

output_file = os.path.join(data_folder, 'pressure_time_series.pdf')
plt.savefig(output_file, format='pdf', dpi=300)  # Save as PDF for better quality
# Show the plot (optional)
plt.show()

#print(all_pressure_data[p_column].mean())


In [ ]:

# 1. Plot time series of temperature measurements for the entire day
plt.figure(figsize=(10, 6))
for column in temperature_columns:
    plt.plot(combined_data['TIMESTAMP'], combined_data[column], label=column)

# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(date_format)

# Set labels, title, and legend with larger font sizes
plt.xlabel('Time (UTC)', fontsize=16)
plt.ylabel('Temperature (°C)', fontsize=16)
plt.title('Time Series of Temperature Measurements for the Whole Day', fontsize=18)
plt.legend(fontsize=14)
plt.grid(True)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()

# Save the plot as a JPEG file
output_file = os.path.join(data_folder, 'temperature_time_series.jpg')
plt.savefig(output_file, format='jpg', dpi=300)
plt.show()


# 2. Plot the dry static energy over the whole day for each height
plt.figure(figsize=(10, 6))
for height in heights:
    plt.plot(all_dse_data['TIMESTAMP'], all_dse_data[f'Dry_Static_Energy_{height}'], label=f'{height}m')

# Format the x-axis to show time in HH:MM format
plt.gca().xaxis.set_major_formatter(date_format)

# Set labels, title, and legend with larger font sizes
plt.xlabel('Time (UTC)', fontsize=16)
plt.ylabel('Dry Static Energy (over heat capacity) (K)', fontsize=16)
plt.title('Dry Static Energy Variation Over the Whole Day', fontsize=18)
plt.legend(title='Height', loc='upper right', fontsize=14, title_fontsize=14)
plt.grid(True)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()

# Save the plot as a JPEG file
output_file = os.path.join(data_folder, 'drystaticenergy_time_series.jpg')
plt.savefig(output_file, format='jpg', dpi=300)
plt.show()


# 3. Plot time series of wind speed measurements for the entire day
plt.figure(figsize=(10, 6))
for column in ws_columns:
    plt.plot(all_ws_data['TIMESTAMP'], all_ws_data[column], label=column)

# Format the x-axis to show time in HH:MM format
plt.gca().xaxis.set_major_formatter(date_format)

# Set labels, title, and legend with larger font sizes
plt.xlabel('Time', fontsize=16)
plt.ylabel('Wind Speed (m/s)', fontsize=16)
plt.title('Time Series of Wind Speed Measurements for the Whole Day', fontsize=18)
plt.legend(fontsize=14)
plt.grid(True)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()

# Save the plot as a JPEG file
output_file = os.path.join(data_folder, 'windspeed_time_series.jpg')
plt.savefig(output_file, format='jpg', dpi=300)
plt.show()


# 4. Plot time series of pressure measurements for the entire day
plt.figure(figsize=(10, 6))
plt.plot(all_pressure_data['TIMESTAMP'], all_pressure_data[p_column], label=p_column)

# Format the x-axis to show time in HH:MM format
plt.gca().xaxis.set_major_formatter(date_format)

# Set labels, title, and legend with larger font sizes
plt.xlabel('Time', fontsize=16)
plt.ylabel('Pressure at 2.99 m (mbar)', fontsize=16)
plt.title('Time Series of Pressure Measurements for the Whole Day', fontsize=18)
plt.legend(fontsize=14)
plt.grid(True)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()

# Save the plot as a JPEG file
output_file = os.path.join(data_folder, 'pressure_time_series.jpg')
plt.savefig(output_file, format='jpg', dpi=300)
plt.show()


In [ ]:

all_pressure_data['rho_air']=all_pressure_data[p_column]*100/(Rd*(all_pressure_data['AirTC_E5568_Avg']+273.15))
all_pressure_data['rho_air_Tv']=all_pressure_data[p_column]*100/(Rd*(all_pressure_data['AirTC_E5568_Avg']+273.15)*(1+(((Rv/Rd)-1)*(all_pressure_data['qv_2.99m']/1000))))
#print(all_pressure_data)
# Plot time series of temperature measurements for the entire day
plt.figure(figsize=(10, 6))

plt.plot(all_pressure_data['TIMESTAMP'], all_pressure_data['rho_air'],label='rho_air from T')
plt.plot(all_pressure_data['TIMESTAMP'], all_pressure_data['rho_air_Tv'],label='rho_air from T_v')
    
    
# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(date_format)

plt.xlabel('Time')
plt.ylabel('rho_air (kg/m^3)')
plt.title('Time Series of Density of Air at 2.99 m')
#plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.legend()
# Save the plot as a JPEG file in the same folder
output_file = os.path.join(data_folder, 'density_air_time_series.pdf')
#plt.savefig(output_file, format='jpg')
plt.savefig(output_file, format='pdf', dpi=300)  # Save as PDF for better quality

# Show the plot (optional)
plt.show()

# Plot specific humidity for each height
plt.figure(figsize=(10, 6))
for height in heights:
    plt.plot(all_qv_data['TIMESTAMP'], all_qv_data[f'qv_{height}m'], label=f'{height}m')
    
plt.gca().xaxis.set_major_formatter(date_format)


plt.xlabel('Time (UTC)')
plt.ylabel('Specific Humidity, q_v (g/kg)')
plt.title('Specific Humidity Variation Over the Whole Day')
plt.legend(title='Height', loc='upper right')

#plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Save the plot
plot_name = 'specific_humidity_time_series.pdf'
plt.savefig(os.path.join(data_folder, plot_name),dpi=300)
#plt.savefig(output_file, format='pdf', dpi=300)  # Save as PDF for better quality

plt.show()

# Plot relative humidity for each height
plt.figure(figsize=(10, 6))
for column in rh_columns:
    plt.plot(all_rh_data['TIMESTAMP'], all_rh_data[column], label=column)#f'Relative Humidity - Height: {height}m')
    
plt.gca().xaxis.set_major_formatter(date_format)


plt.xlabel('Time (UTC)')
plt.ylabel('Relative Humidity (%)')
plt.title('Relative Humidity Variation Over the Whole Day')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Save the plot
plot_name = 'relative_humidity_time_series.pdf'
plt.savefig(os.path.join(data_folder, plot_name),dpi=300)

plt.show()

In [ ]:

# Define date format for x-axis
date_format = DateFormatter('%H:%M')

# 1. Plot time series of air density measurements
all_pressure_data['rho_air'] = all_pressure_data[p_column]*100/(Rd*(all_pressure_data['AirTC_E5568_Avg']+273.15))
all_pressure_data['rho_air_Tv'] = all_pressure_data[p_column]*100/(Rd*(all_pressure_data['AirTC_E5568_Avg']+273.15)*(1+(((Rv/Rd)-1)*(all_pressure_data['qv_2.99m']/1000))))

plt.figure(figsize=(10, 6))
plt.plot(all_pressure_data['TIMESTAMP'], all_pressure_data['rho_air'], label='rho_air from T')
plt.plot(all_pressure_data['TIMESTAMP'], all_pressure_data['rho_air_Tv'], label='rho_air from T_v')

# Format the x-axis to show time in HH:MM format
plt.gca().xaxis.set_major_formatter(date_format)

# Set labels, title, and legend with larger font sizes
plt.xlabel('Time (UTC)', fontsize=16)
plt.ylabel('rho_air (kg/m³)', fontsize=16)
plt.title('Time Series of Density of Air at 2.99 m', fontsize=18)
plt.legend(fontsize=14)
plt.grid(True)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()

# Save the plot as a JPEG file
output_file = os.path.join(data_folder, 'density_air_time_series.jpg')
plt.savefig(output_file, format='jpg', dpi=300)
plt.show()


# 2. Plot specific humidity for each height
plt.figure(figsize=(10, 6))
for height in heights:
    plt.plot(all_qv_data['TIMESTAMP'], all_qv_data[f'qv_{height}m'], label=f'{height}m')

plt.gca().xaxis.set_major_formatter(date_format)

# Set labels, title, and legend with larger font sizes
plt.xlabel('Time (UTC)', fontsize=16)
plt.ylabel('Specific Humidity, q_v (g/kg)', fontsize=16)
plt.title('Specific Humidity Variation Over the Whole Day', fontsize=18)
plt.legend(title='Height', loc='upper right', fontsize=14, title_fontsize=14)
plt.grid(True)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()

# Save the plot as a JPEG file
plot_name = 'specific_humidity_time_series.jpg'
plt.savefig(os.path.join(data_folder, plot_name), format='jpg', dpi=300)
plt.show()


# 3. Plot relative humidity for each height
plt.figure(figsize=(10, 6))
for column in rh_columns:
    plt.plot(all_rh_data['TIMESTAMP'], all_rh_data[column], label=column)

plt.gca().xaxis.set_major_formatter(date_format)

# Set labels, title, and legend with larger font sizes
plt.xlabel('Time (UTC)', fontsize=16)
plt.ylabel('Relative Humidity (%)', fontsize=16)
plt.title('Relative Humidity Variation Over the Whole Day', fontsize=18)
plt.legend(fontsize=14)
plt.grid(True)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()

# Save the plot as a JPEG file
plot_name = 'relative_humidity_time_series.jpg'
plt.savefig(os.path.join(data_folder, plot_name), format='jpg', dpi=300)
plt.show()


In [ ]:

# Convert TIMESTAMP to datetime if it's not already
all_pressure_data['TIMESTAMP'] = pd.to_datetime(all_pressure_data['TIMESTAMP'], errors='coerce')

# Set the TIMESTAMP column as the index
all_pressure_data.set_index('TIMESTAMP', inplace=True)

# Select only numeric columns for resampling
numeric_columns = all_pressure_data.select_dtypes(include='number').columns
all_pressure_data_numeric = all_pressure_data[numeric_columns]

# Resample the data to 10-minute intervals and calculate the mean for each interval
all_pressure_data_10min = all_pressure_data_numeric.resample('10T').mean()

# Reset the index to make TIMESTAMP a column again
all_pressure_data_10min.reset_index(inplace=True)

# Display the resulting DataFrame
print(all_pressure_data_10min)


In [ ]:
print(all_pressure_data_10min.columns)

### Solar/IR

In [ ]:
# Directory containing the sonic data files
# ⚠️ Edit this path to match your local setup before running.
data_folder_sonic = f"C:\\path\\to\\your\\radiometer\\{month_str}\\{date_str}"


# Get a list of all .dat files in the folder
data_files_sonic = [file for file in os.listdir(data_folder_sonic) if file.endswith('.dat')]

# Initialize an empty list to store the short wave down radiation data for each file
all_data_swr = []

all_data_swr_net=[]

all_data_swr_up = []


all_data_lwr_net=[]

all_data_lwr_down = []

all_data_lwr_up = []

all_data_albedo=[] #albedo at surface, Swup/Swdown
filtered_albedo = [] # albedo values when Swdown > 10


# Loop through each file in the folder
for file in data_files_sonic:
    # Construct the full file path
    file_path = os.path.join(data_folder_sonic, file)
    
    # Read the data from the file
    data = pd.read_csv(file_path, skiprows=1, delimiter=',', encoding='latin1')
    
    # Convert TIMESTAMP column to datetime format with error handling
    data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'], errors='coerce')
    
    # Drop rows with NaT values in the TIMESTAMP column
    data.dropna(subset=['TIMESTAMP'], inplace=True)
    
    # Select relevant columns containing short wave down radiation data
    swr_column = 'SR15D1Dn_Irr'  # Adjust column name if needed
    swr_up_column = 'SR15D1Up_Irr'
    swr_net_column = 'NetRs'
    # Select relevant columns containing long wave down radiation data

    lwr_net_column= 'NetRl'
    lwr_down_column= 'IR20Dn'
    lwr_up_column= 'IR20Up'
    
    
    albedo_column = 'Albedo'
    
    # Convert the column to numeric type
    data[swr_column] = pd.to_numeric(data[swr_column], errors='coerce')
    
    
    # Append the data to the list
    all_data_swr.append(data)
    
    # Plot and save the time series of short wave down radiation measurements for each file
    plt.figure(figsize=(10, 6))
    plt.plot(data['TIMESTAMP'], data[swr_column], label='Short Wave Down Radiation')
    plt.xlabel('Time')
    plt.ylabel('Short Wave Down Radiation')
    plt.title(f'Time Series of Short Wave Down Radiation - {file}')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(data_folder_sonic, f'{os.path.splitext(file)[0]}_swr_plot.jpg'))
    plt.close()
    
    # Convert the column to numeric type    
    data[lwr_net_column] = pd.to_numeric(data[lwr_net_column], errors='coerce')

    all_data_lwr_net.append(data)
    # Plot and save the time series of short wave down radiation measurements for each file
    plt.figure(figsize=(10, 6))
    plt.plot(data['TIMESTAMP'], data[lwr_net_column], label='Long Wave Net Radiation')
    plt.xlabel('Time')
    plt.ylabel('Long Wave Net Radiation, LW_Net[w/m^2]')
    plt.title(f'Time Series of Net Long Wave Radiation - {file}')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(data_folder_sonic, f'{os.path.splitext(file)[0]}_lwr_net_plot.jpg'))
    plt.close()
    
    # Convert the column to numeric type    
    data[lwr_down_column] = pd.to_numeric(data[lwr_down_column], errors='coerce')

    all_data_lwr_down.append(data)
    
    # Convert the column to numeric type    
    data[lwr_up_column] = pd.to_numeric(data[lwr_up_column], errors='coerce')

    all_data_lwr_up.append(data)
    
     
    # Convert the column to numeric type    
    data[swr_up_column] = pd.to_numeric(data[swr_up_column], errors='coerce')

    all_data_swr_up.append(data)
    
    # Convert the column to numeric type    
    data[swr_net_column] = pd.to_numeric(data[swr_net_column], errors='coerce')

    all_data_swr_net.append(data)
    
    ## albedo 
    data[albedo_column]= pd.to_numeric(data[albedo_column], errors='coerce')
    
    all_data_albedo.append(data)
    # Filter albedo values where short wave down radiation is larger than 10
    # Filter albedo values where short wave down radiation is larger than 10
    filtered_albedo_data = data[data[swr_column] > 10][[albedo_column, 'TIMESTAMP']]
    filtered_albedo.append(filtered_albedo_data)

# Combine data from all files
all_data_combined_swr = pd.concat(all_data_swr, ignore_index=True)

all_data_combined_lwr_net = pd.concat(all_data_lwr_net, ignore_index=True)

all_data_combined_lwr_down = pd.concat(all_data_lwr_down, ignore_index=True)

all_data_combined_lwr_up = pd.concat(all_data_lwr_up, ignore_index=True)


all_data_combined_swr_up = pd.concat(all_data_swr_up, ignore_index=True)

all_data_combined_swr_net = pd.concat(all_data_swr_net, ignore_index=True)


all_data_combined_albedo= pd.concat(all_data_albedo, ignore_index=True)
filtered_albedo_combined = pd.concat(filtered_albedo, ignore_index=True)


In [ ]:
'''
# Plot and save the combined time series of albedo measurements
plt.figure(figsize=(10, 6))
plt.plot(filtered_albedo_combined['TIMESTAMP'], 1/filtered_albedo_combined[albedo_column], label='Albedo')

#plt.plot(all_data_combined_albedo['TIMESTAMP'], all_data_combined_swr_up[swr_up_column]/all_data_combined_swr[swr_column], label='Albedo calc')

# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(date_format)

# Set y-axis limits from -0.1 to 1.1
#plt.ylim(-1.1, 0.5)

plt.xlabel('Time')
plt.ylabel('Surface Albedo')
plt.title('Combined Time Series of Surface ALbedo')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(data_folder_sonic, 'combined_albedo_plot.pdf'),dpi=300)

plt.show()

# Plot and save the combined time series of short wave down radiation measurements
plt.figure(figsize=(10, 6))
plt.plot(all_data_combined_swr['TIMESTAMP'], all_data_combined_swr[swr_column], label='Short Wave Down Radiation')

# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(date_format)
plt.xlabel('Time')
plt.ylabel('Short Wave Down Radiation, SW_dn [W/m^2]')
plt.title('Combined Time Series of Short Wave Down Radiation [W/m^2]')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(data_folder_sonic, 'combined_swr_plot.jpg'))
plt.show()

# Create a figure with four subplots arranged in a 2x2 grid
fig, axs = plt.subplots(2, 2, figsize=(14, 12), sharex=True)

# Plot net long wave radiation
axs[0, 0].plot(all_data_combined_swr_net['TIMESTAMP'], all_data_combined_swr_net[swr_net_column], label='Net Short Wave Radiation')
axs[0, 0].set_ylabel('Net SW Radiation [W/m^2]')
axs[0, 0].set_title('Net Short Wave Radiation')
axs[0, 0].legend()
axs[0, 0].grid(True)

# Plot long wave radiation up
axs[0, 1].plot(all_data_combined_swr_up['TIMESTAMP'], all_data_combined_swr_up[swr_up_column], label='Short Wave Radiation Up, SW_up', color='orange')
axs[0, 1].set_ylabel('SW Radiation Up [W/m^2]')
axs[0, 1].set_title('Short Wave Radiation Up')
axs[0, 1].legend()
axs[0, 1].grid(True)

# Plot long wave radiation down
axs[1, 0].plot(all_data_combined_swr['TIMESTAMP'], all_data_combined_swr[swr_column], label='Short Wave Radiation Down, SW_dn', color='green')
axs[1, 0].set_ylabel('SW Radiation Down [W/m^2]')
axs[1, 0].set_xlabel('Time')
axs[1, 0].set_title('Short Wave Radiation Down')
axs[1, 0].legend()
axs[1, 0].grid(True)

# Plot all values combined
axs[1, 1].plot(all_data_combined_swr_net['TIMESTAMP'], all_data_combined_swr_net[swr_net_column], label='Net Short Wave Radiation')
axs[1, 1].plot(all_data_combined_swr_up['TIMESTAMP'], all_data_combined_swr_up[swr_up_column], label='Short Wave Radiation Up, SW_up', color='orange')
axs[1, 1].plot(all_data_combined_swr['TIMESTAMP'], all_data_combined_swr[swr_column], label='Short Wave Radiation Down, SW_dn', color='green')
axs[1, 1].set_xlabel('Time')
axs[1, 1].set_ylabel('SW Radiation [W/m^2]')
axs[1, 1].set_title('Combined Short Wave Radiation')
axs[1, 1].legend()
axs[1, 1].grid(True)

# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
for ax in axs.flat:
    ax.xaxis.set_major_formatter(date_format)
    ax.tick_params(rotation=45)

plt.tight_layout()
plt.savefig(os.path.join(data_folder_sonic, 'combined_swr_plots.pdf'),dpi=300)
plt.show()

# Plot and save the combined time series of net long wave down radiation measurements
plt.figure(figsize=(10, 6))
plt.plot(all_data_combined_lwr_net['TIMESTAMP'], all_data_combined_lwr_net[lwr_net_column], label='Net Long Wave Radiation')

plt.plot(all_data_combined_lwr_up['TIMESTAMP'], all_data_combined_lwr_up[lwr_up_column], label='Long Wave Radiation Up, LW_up')

plt.plot(all_data_combined_lwr_down['TIMESTAMP'], all_data_combined_lwr_down[lwr_down_column], label='Long Wave Radiation Down, LW_dn')

# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(date_format)
plt.xlabel('Time')
plt.ylabel('Net Long Wave Radiation, LW_net [W/m^2]')
plt.title('Combined Time Series of Long Wave Radiation [W/m^2]')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(data_folder_sonic, 'combined_lwr_plot.jpg'))
plt.show()

# Create a figure with four subplots arranged in a 2x2 grid
fig, axs = plt.subplots(2, 2, figsize=(14, 12), sharex=True)

# Plot net long wave radiation
axs[0, 0].plot(all_data_combined_lwr_net['TIMESTAMP'], all_data_combined_lwr_net[lwr_net_column], label='Net Long Wave Radiation')
axs[0, 0].set_ylabel('Net LW Radiation [W/m^2]')
axs[0, 0].set_title('Net Long Wave Radiation')
axs[0, 0].legend()
axs[0, 0].grid(True)

# Plot long wave radiation up
axs[0, 1].plot(all_data_combined_lwr_up['TIMESTAMP'], all_data_combined_lwr_up[lwr_up_column], label='Long Wave Radiation Up, LW_up', color='orange')
axs[0, 1].set_ylabel('LW Radiation Up [W/m^2]')
axs[0, 1].set_title('Long Wave Radiation Up')
axs[0, 1].legend()
axs[0, 1].grid(True)

# Plot long wave radiation down
axs[1, 0].plot(all_data_combined_lwr_down['TIMESTAMP'], all_data_combined_lwr_down[lwr_down_column], label='Long Wave Radiation Down, LW_dn', color='green')
axs[1, 0].set_ylabel('LW Radiation Down [W/m^2]')
axs[1, 0].set_xlabel('Time')
axs[1, 0].set_title('Long Wave Radiation Down')
axs[1, 0].legend()
axs[1, 0].grid(True)

# Plot all values combined
axs[1, 1].plot(all_data_combined_lwr_net['TIMESTAMP'], all_data_combined_lwr_net[lwr_net_column], label='Net Long Wave Radiation')
axs[1, 1].plot(all_data_combined_lwr_up['TIMESTAMP'], all_data_combined_lwr_up[lwr_up_column], label='Long Wave Radiation Up, LW_up', color='orange')
axs[1, 1].plot(all_data_combined_lwr_down['TIMESTAMP'], all_data_combined_lwr_down[lwr_down_column], label='Long Wave Radiation Down, LW_dn', color='green')
axs[1, 1].set_xlabel('Time')
axs[1, 1].set_ylabel('LW Radiation [W/m^2]')
axs[1, 1].set_title('Combined Long Wave Radiation')
axs[1, 1].legend()
axs[1, 1].grid(True)

# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
for ax in axs.flat:
    ax.xaxis.set_major_formatter(date_format)
    ax.tick_params(rotation=45)

plt.tight_layout()
plt.savefig(os.path.join(data_folder_sonic, 'combined_lwr_plots.pdf'),dpi=300)
plt.show()
'''

In [ ]:
# Plot and save the combined time series of albedo measurements
plt.figure(figsize=(10, 6))
plt.plot(filtered_albedo_combined['TIMESTAMP'], 1/filtered_albedo_combined[albedo_column], 
         label='Albedo', color='blue')

# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(date_format)

# Set y-axis limits if needed (optional)
# plt.ylim(-1.1, 0.5)

# Set title and labels with larger font sizes
plt.xlabel('Time', fontsize=16)
plt.ylabel('Surface Albedo', fontsize=16)
plt.title('Combined Time Series of Surface Albedo', fontsize=18)
plt.legend(fontsize=14)
plt.grid(True)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()
plt.savefig(os.path.join(data_folder_sonic, 'combined_albedo_plot.png'), dpi=300)
plt.show()

# Plot and save the combined time series of short wave down radiation measurements
plt.figure(figsize=(10, 6))
plt.plot(all_data_combined_swr['TIMESTAMP'], all_data_combined_swr[swr_column], 
         label='Short Wave Down Radiation', color='blue')

# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(date_format)

# Set title and labels with larger font sizes
plt.xlabel('Time', fontsize=16)
plt.ylabel('Short Wave Down Radiation, SW_dn [W/m²]', fontsize=16)
plt.title('Combined Time Series of Short Wave Down Radiation [W/m²]', fontsize=18)
plt.legend(fontsize=14)
plt.grid(True)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()
plt.savefig(os.path.join(data_folder_sonic, 'combined_swr_plot.png'), dpi=300)
plt.show()

# Create a figure with four subplots arranged in a 2x2 grid
fig, axs = plt.subplots(2, 2, figsize=(14, 12), sharex=True)

# Plot net long wave radiation
axs[0, 0].plot(all_data_combined_swr_net['TIMESTAMP'], all_data_combined_swr_net[swr_net_column], 
                label='Net Short Wave Radiation', color='blue')
axs[0, 0].set_ylabel('Net SW Radiation [W/m²]', fontsize=14)
axs[0, 0].set_title('Net Short Wave Radiation', fontsize=16)
axs[0, 0].legend(fontsize=12)
axs[0, 0].grid(True)

# Plot long wave radiation up
axs[0, 1].plot(all_data_combined_swr_up['TIMESTAMP'], all_data_combined_swr_up[swr_up_column], 
                label='Short Wave Radiation Up, SW_up', color='orange')
axs[0, 1].set_ylabel('SW Radiation Up [W/m²]', fontsize=14)
axs[0, 1].set_title('Short Wave Radiation Up', fontsize=16)
axs[0, 1].legend(fontsize=12)
axs[0, 1].grid(True)

# Plot long wave radiation down
axs[1, 0].plot(all_data_combined_swr['TIMESTAMP'], all_data_combined_swr[swr_column], 
                label='Short Wave Radiation Down, SW_dn', color='green')
axs[1, 0].set_ylabel('SW Radiation Down [W/m²]', fontsize=14)
axs[1, 0].set_xlabel('Time', fontsize=16)
axs[1, 0].set_title('Short Wave Radiation Down', fontsize=16)
axs[1, 0].legend(fontsize=12)
axs[1, 0].grid(True)

# Plot all values combined
axs[1, 1].plot(all_data_combined_swr_net['TIMESTAMP'], all_data_combined_swr_net[swr_net_column], 
                label='Net Short Wave Radiation', color='blue')
axs[1, 1].plot(all_data_combined_swr_up['TIMESTAMP'], all_data_combined_swr_up[swr_up_column], 
                label='Short Wave Radiation Up, SW_up', color='orange')
axs[1, 1].plot(all_data_combined_swr['TIMESTAMP'], all_data_combined_swr[swr_column], 
                label='Short Wave Radiation Down, SW_dn', color='green')
axs[1, 1].set_xlabel('Time', fontsize=16)
axs[1, 1].set_ylabel('SW Radiation [W/m²]', fontsize=14)
axs[1, 1].set_title('Combined Short Wave Radiation', fontsize=16)
axs[1, 1].legend(fontsize=12)
axs[1, 1].grid(True)

# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
for ax in axs.flat:
    ax.xaxis.set_major_formatter(date_format)
    ax.tick_params(rotation=45)

plt.tight_layout()
plt.savefig(os.path.join(data_folder_sonic, 'combined_swr_plots.png'), dpi=300)
plt.show()

# Plot and save the combined time series of net long wave down radiation measurements
plt.figure(figsize=(10, 6))
plt.plot(all_data_combined_lwr_net['TIMESTAMP'], all_data_combined_lwr_net[lwr_net_column], 
         label='Net Long Wave Radiation', color='blue')
plt.plot(all_data_combined_lwr_up['TIMESTAMP'], all_data_combined_lwr_up[lwr_up_column], 
         label='Long Wave Radiation Up, LW_up', color='orange')
plt.plot(all_data_combined_lwr_down['TIMESTAMP'], all_data_combined_lwr_down[lwr_down_column], 
         label='Long Wave Radiation Down, LW_dn', color='green')

# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(date_format)

# Set title and labels with larger font sizes
plt.xlabel('Time', fontsize=16)
plt.ylabel('Net Long Wave Radiation, LW_net [W/m²]', fontsize=16)
plt.title('Combined Time Series of Long Wave Radiation [W/m²]', fontsize=18)
plt.legend(fontsize=14)
plt.grid(True)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()
plt.savefig(os.path.join(data_folder_sonic, 'combined_lwr_plot.png'), dpi=300)
plt.show()

# Create a figure with four subplots arranged in a 2x2 grid for long wave radiation
fig, axs = plt.subplots(2, 2, figsize=(14, 12), sharex=True)

# Plot net long wave radiation
axs[0, 0].plot(all_data_combined_lwr_net['TIMESTAMP'], all_data_combined_lwr_net[lwr_net_column], 
                label='Net Long Wave Radiation', color='blue')
axs[0, 0].set_ylabel('Net LW Radiation [W/m²]', fontsize=14)
axs[0, 0].set_title('Net Long Wave Radiation', fontsize=16)
axs[0, 0].legend(fontsize=12)
axs[0, 0].grid(True)

# Plot long wave radiation up
axs[0, 1].plot(all_data_combined_lwr_up['TIMESTAMP'], all_data_combined_lwr_up[lwr_up_column], 
                label='Long Wave Radiation Up, LW_up', color='orange')
axs[0, 1].set_ylabel('LW Radiation Up [W/m²]', fontsize=14)
axs[0, 1].set_title('Long Wave Radiation Up', fontsize=16)
axs[0, 1].legend(fontsize=12)
axs[0, 1].grid(True)

# Plot long wave radiation down
axs[1, 0].plot(all_data_combined_lwr_down['TIMESTAMP'], all_data_combined_lwr_down[lwr_down_column], 
                label='Long Wave Radiation Down, LW_dn', color='green')
axs[1, 0].set_ylabel('LW Radiation Down [W/m²]', fontsize=14)
axs[1, 0].set_xlabel('Time', fontsize=16)
axs[1, 0].set_title('Long Wave Radiation Down', fontsize=16)
axs[1, 0].legend(fontsize=12)
axs[1, 0].grid(True)

# Plot all values combined
axs[1, 1].plot(all_data_combined_lwr_net['TIMESTAMP'], all_data_combined_lwr_net[lwr_net_column], 
                label='Net Long Wave Radiation', color='blue')
axs[1, 1].plot(all_data_combined_lwr_up['TIMESTAMP'], all_data_combined_lwr_up[lwr_up_column], 
                label='Long Wave Radiation Up, LW_up', color='orange')
axs[1, 1].plot(all_data_combined_lwr_down['TIMESTAMP'], all_data_combined_lwr_down[lwr_down_column], 
                label='Long Wave Radiation Down, LW_dn', color='green')
axs[1, 1].set_xlabel('Time', fontsize=16)
axs[1, 1].set_ylabel('LW Radiation [W/m²]', fontsize=14)
axs[1, 1].set_title('Combined Long Wave Radiation', fontsize=16)
axs[1, 1].legend(fontsize=12)
axs[1, 1].grid(True)

# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
for ax in axs.flat:
    ax.xaxis.set_major_formatter(date_format)
    ax.tick_params(rotation=45)

plt.tight_layout()
plt.savefig(os.path.join(data_folder_sonic, 'combined_lwr_plots.png'), dpi=300)
plt.show()


In [ ]:
# Calculate surface temperature from LWR up data
all_data_combined_lwr_up['Surface_Temperature'] = (all_data_combined_lwr_up['IR20Up'] / sigma) ** 0.25
#all_data_combined_lwr_up = pd.concat(all_data_lwr_up, ignore_index=True)

# Plot and save the time series of surface temperature
plt.figure(figsize=(10, 6))
plt.plot(all_data_combined_lwr_up['TIMESTAMP'], all_data_combined_lwr_up['Surface_Temperature'], label='Surface Temperature')
plt.xlabel('Time')
plt.ylabel('Surface Temperature (K)')
plt.title('Time Series of Surface Temperature')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(data_folder_sonic, 'surface_temperature_plot.jpg'))
plt.show()
#print(all_data_combined_lwr_up)
#print("Surface temperature calculation completed and plot saved.")

In [ ]:
# Resample to 1-minute intervals and calculate the mean
#all_data_combined_lwr_up = pd.concat(all_data_lwr_up, ignore_index=True)

all_data_combined_lwr_up['TIMESTAMP'] = pd.to_datetime(all_data_combined_lwr_up['TIMESTAMP'], errors='coerce')
all_data_combined_lwr_up.set_index('TIMESTAMP', inplace=True)


#print(all_data_combined_lwr_up)
resampled_surface_temp = all_data_combined_lwr_up['Surface_Temperature'].resample('1T').mean()
print(resampled_surface_temp)


# Plot both temperatures
plt.figure(figsize=(10, 6))
plt.plot(combined_data['TIMESTAMP'],combined_data['AirTC_E5567_Avg']+273.15, label='T_2m (K) from mast')
plt.plot(resampled_surface_temp, label='T_srf from LW_up (K)')  # Convert to °C
plt.xlabel('Time')
plt.ylabel('Temperature (°C)')
plt.title('Time Series of T_2m and Surface Temperature')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(data_folder_sonic, 'T2m_and_surface_temperature_plot.jpg'))
plt.show()
plt.close()

In [ ]:
#def calculate_clear_sky_sw_down_again(latitude, longitude, timestamps):
    # Create location object
 #   location = pvlib.location.Location(latitude, longitude)
    
    # Calculate clear-sky GHI using Ineichen model
  #  clearsky = location.get_clearsky(timestamps, model='ineichen') #Ineichen model
    
    # Extract clear-sky GHI values
   # clear_sky_sw_down = clearsky['ghi']
    
    #return clear_sky_sw_down
def calculate_clear_sky_sw_down_again(latitude, longitude, timestamps):
    # Create location object
    location = pvlib.location.Location(latitude, longitude)
    
    # Calculate clear-sky GHI using Ineichen model
    clearsky = location.get_clearsky(timestamps, model='ineichen')
    
    # Extract timestamps and clear-sky GHI values
    clear_sky_sw_down = clearsky['ghi']
    timestamps = clear_sky_sw_down.index
    
    # Create DataFrame with columns 'timestamp' and 'ghi'
    clear_sky_df = pd.DataFrame({
        'TIMESTAMP': timestamps,
        'ghi': clear_sky_sw_down.values
    })
    
    return clear_sky_df
# Example usage
latitude = 52.6324  # Latitude of Alkmaar, Netherlands
longitude = 4.7534  # Longitude of Alkmaar, Netherlands
#timestamps = pd.date_range(start='2024-05-02 00:00:00', end='2024-05-02 23:59:59', freq='1S')  # Frequency set to 1 second
timestamps = pd.to_datetime(all_data_combined_swr['TIMESTAMP'], errors='coerce')# Use actual data timestamps
# Convert timestamps to datetime index
timestamps = pd.DatetimeIndex(timestamps)

# Call the function to calculate clear-sky SW down radiation
clear_sky_sw_down = calculate_clear_sky_sw_down_again(latitude, longitude, timestamps)


# Plot the clear-sky SW down radiation
plt.figure(figsize=(10, 6))
# Plot measured short wave down radiation
plt.plot(all_data_combined_swr['TIMESTAMP'], all_data_combined_swr[swr_column], 
         label='Measured SW Down Radiation', color='blue')

plt.plot(clear_sky_sw_down['TIMESTAMP'], clear_sky_sw_down['ghi'], 
         label='Clear-Sky SW Down Radiation (pvlib)', color='red')

# Format the x-axis to show time in HH:MM format
date_format = DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(date_format)

# Set title and labels with larger font sizes
plt.xlabel('Time', fontsize=16)
plt.ylabel('SW Down Radiation (W/m²)', fontsize=16)
plt.title('SW Down Radiation - May 11, 2024', fontsize=18)
plt.legend(fontsize=14)
plt.grid(True)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()
plt.savefig(os.path.join(data_folder_sonic, 'combined_swr_plot_with_clear_sky.jpg'))
plt.show()

# Plot the clear-sky GHI
plt.figure(figsize=(10, 6))
# Plot measured short wave down radiation
plt.plot(all_data_combined_swr['TIMESTAMP'], all_data_combined_swr[swr_column], 
         label='Measured SW Down Radiation', color='blue')

plt.plot(clear_sky_sw_down['TIMESTAMP'], clear_sky_sw_down['ghi'], 
         label='Clear-Sky SW Down Radiation (pvlib-Ineichen)', color='green')

# Format the x-axis to show time in HH:MM format
plt.gca().xaxis.set_major_formatter(date_format)

# Set title and labels with larger font sizes
plt.xlabel('Time', fontsize=16)
plt.ylabel('SW Down Radiation (W/m²)', fontsize=16)
plt.title('SW Down Radiation - May 23, 2024', fontsize=18)
plt.legend(fontsize=14)
plt.grid(True)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()
#plt.savefig(os.path.join(data_folder_sonic, 'combined_swr_plot_with_clear_sky_ghi.jpg'))
plt.show()



# Step 3: Filter non-zero clear sky GHI values
non_zero_clear_sky = clear_sky_sw_down[clear_sky_sw_down['ghi'] > 0]



merged_data_csi= pd.merge(non_zero_clear_sky, all_data_combined_swr, on='TIMESTAMP', how='inner')

# Calculate Clear Sky Index (CSI) using 'SR15D1Dn_Irr' and 'clear_sky_ghi'
merged_data_csi['CSI'] = merged_data_csi['SR15D1Dn_Irr'] / merged_data_csi['ghi']
#print(merged_data)

# Plotting CSI against time
plt.figure(figsize=(10, 6))
plt.plot(merged_data_csi['TIMESTAMP'], merged_data_csi['CSI'], 
         marker='o', linestyle='-', color='b', markersize=1)

# Set title and labels with larger font sizes
plt.title('Clear Sky Index (CSI) vs Time', fontsize=18)
plt.gca().xaxis.set_major_formatter(date_format)
plt.xlabel('Time', fontsize=16)
plt.ylabel('Clear Sky Index (CSI)', fontsize=16)
plt.ylim(0, 1.5)  # Set y-axis limits from 0 to 1

plt.grid(True)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()
#plt.savefig(os.path.join(data_folder_sonic, 'csi_plot.jpg'))
plt.show()
 #Assuming all_data_combined_lwr_net, all_data_combined_lwr_up, all_data_combined_lwr_down, 
# combined_data, all_data_combined_swr, merged_data_csi, and all_qv_data are appropriately defined

# Create a figure with a 2x3 grid of plots
fig, axs = plt.subplots(2, 3, figsize=(18, 12))

# Plot 1: Combined Long Wave Radiation
axs[1, 0].plot(all_data_combined_lwr_net['TIMESTAMP'], all_data_combined_lwr_net[lwr_net_column], label='Net Long Wave Radiation')
axs[1, 0].set_xlabel('Time')
axs[1, 0].set_ylabel('LW Radiation [W/m^2]')
axs[1, 0].set_title('Net Long Wave Radiation')
axs[1, 0].legend()
axs[1, 0].grid(True)
axs[1, 0].tick_params(axis='x', rotation=45)

# Plot 2: Long Wave Radiation Up
axs[0, 0].plot(all_data_combined_lwr_net['TIMESTAMP'], all_data_combined_lwr_net['IR20Up'], label='Long Wave Radiation Up, LW_up', color='orange')
axs[0, 0].set_ylabel('LW Radiation Up [W/m^2]')
axs[0, 0].set_title('Long Wave Radiation Up')
axs[0, 0].legend()
axs[0, 0].grid(True)
axs[0, 0].tick_params(axis='x', rotation=45)
# Plot 3: Temperature Series
axs[0, 1].plot(combined_data['TIMESTAMP'], combined_data['AirTC_E5567_Avg'] + 273.15, label='T_2m (K) from mast')
axs[0, 1].plot(resampled_surface_temp.index, resampled_surface_temp.values, label='T_srf from LW_up (K)')  # Convert to °C
axs[0, 1].set_xlabel('Time')
axs[0, 1].set_ylabel('Temperature (K)')
axs[0, 1].set_title('Time Series of T_2m and Surface Temperature')
axs[0, 1].legend()
axs[0, 1].grid(True)
axs[0, 1].tick_params(axis='x', rotation=45)

# Plot 4: SW Down Radiation Comparison
axs[1, 1].plot(all_data_combined_swr['TIMESTAMP'], all_data_combined_swr[swr_column], label='Measured SW Down Radiation', color='blue')
axs[1, 1].plot(clear_sky_sw_down['TIMESTAMP'], clear_sky_sw_down['ghi'], label='Clear-Sky SW Down Radiation (pvlib-Ineichen)', color='green')
axs[1, 1].set_xlabel('Time')
axs[1, 1].set_ylabel('SW Down Radiation (W/m²)')
axs[1, 1].set_title('SW Down Radiation Comparison')
axs[1, 1].legend()
axs[1, 1].grid(True)
axs[1, 1].tick_params(axis='x', rotation=45)

# Plot 5: Clear Sky Index (CSI)
axs[0, 2].plot(merged_data_csi['TIMESTAMP'], merged_data_csi['CSI'], marker='o', linestyle='-', color='b', markersize=1)
axs[0, 2].set_title('Clear Sky Index (CSI) vs Time')
axs[0, 2].set_xlabel('Time')
axs[0, 2].set_ylabel('Clear Sky Index (CSI)')
axs[0, 2].set_ylim(0, 1.5)  # Set y-axis limits from 0 to 1.5
axs[0, 2].grid(True)
axs[0, 2].tick_params(axis='x', rotation=45)

# Plot 6: Specific Humidity Variation
axs[1, 2].set_title('Specific Humidity Variation Over the Whole Day')
axs[1, 2].set_xlabel('Time')
axs[1, 2].set_ylabel('Specific Humidity (g/kg)')
axs[1, 2].grid(True)
axs[1, 2].tick_params(axis='x', rotation=45)

for height in heights:
    axs[1, 2].plot(all_qv_data['TIMESTAMP'], all_qv_data[f'qv_{height}m'], label=f'Specific Humidity - Height: {height}m')

axs[1, 2].legend()

plt.tight_layout()

# Save the plot
plt.savefig(os.path.join(data_folder_sonic, 'first_combined_plots_grid.png'),dpi=300)

plt.show()

In [ ]:
# (Keep your existing data-loading steps here, e.g. all_data_combined_swr and clear_sky_sw_down.)

# Create a larger figure
plt.figure(figsize=(8, 6))

# Plot measured SW down radiation with a thicker line
plt.plot(
    all_data_combined_swr['TIMESTAMP'],
    all_data_combined_swr[swr_column],
    label='Measured SW_dn',
    color='blue',
    linewidth=2.0  # thicker line
)

# Plot clear-sky model prediction with a thicker line
plt.plot(
    clear_sky_sw_down['TIMESTAMP'],
    clear_sky_sw_down['ghi'],
    label='Clear‐Sky SW_dn \n(pvlib‐Ineichen)',
    color='green',
    linewidth=2.0  # thicker line
)

# Get current Axes object
ax = plt.gca()

# 1) Thicken all four spines (axis lines)
for spine in ax.spines.values():
    spine.set_linewidth(1.5)

# 2) Increase tick size and thickness
ax.tick_params(axis='both', which='major', labelsize=12, width=1.5, length=6)
ax.tick_params(axis='both', which='minor', width=1.0, length=4)

# 3) Format x‐axis to show hours:minutes
date_format = mdates.DateFormatter('%H:%M')
ax.xaxis.set_major_formatter(date_format)

# (Optional) If you want to have a major tick every hour:
#ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
# And maybe a minor tick every 30 minutes:
#ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=30))
# 4) Format x‐axis to show time in HH:MM
#date_format = mdates.DateFormatter('%H:%M')
ax.xaxis.set_major_formatter(date_format)
ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=30))
# 4) Labels and title with larger font
ax.set_xlabel('Time', fontsize=16, fontweight='bold')
ax.set_ylabel('SW Down Radiation (W/m²)', fontsize=16, fontweight='bold')
ax.set_title('SW Down Radiation', fontsize=18, fontweight='bold')

# 5) Increase legend font & frame linewidth
legend = ax.legend(fontsize=14, frameon=True,loc='upper right')
legend.get_frame().set_linewidth(1.5)

# 6) Thicker grid lines (optional: use linestyle='–' for dashed)
ax.grid(True, which='major', linestyle='--', linewidth=0.8, alpha=0.7)
ax.grid(True, which='minor', linestyle=':', linewidth=0.5, alpha=0.5)

# 7) Rotate x‐tick labels for readability
plt.xticks(rotation=45)

# 8) Tight layout and save
plt.tight_layout()
output_path = os.path.join(data_folder_sonic, 'combined_swr_plot_with_clear_sky_ghi.jpg')
plt.savefig(output_path, dpi=300)  # save at high resolution for report
plt.show()


In [ ]:
# (Assume merged_data_csi and date_format are already defined.)

plt.figure(figsize=(8, 6))

ax = plt.gca()

# 1) Plot CSI with bolder line and slightly larger markers
ax.plot(
    merged_data_csi['TIMESTAMP'],
    merged_data_csi['CSI'],
    #marker='o',
    linestyle='-',
    color='b',
    #markersize=4,      # slightly larger markers
    linewidth=1.5,     # bolder line
    label='CSI'
)

# 2) Thicken spines (axis lines)
for spine in ax.spines.values():
    spine.set_linewidth(1.5)

# 3) Tick parameters for major & minor ticks
ax.tick_params(axis='both', which='major', labelsize=12, width=1.5, length=6)
ax.tick_params(axis='both', which='minor', width=1.0, length=4)

# 4) Format x‐axis with HH:MM
ax.xaxis.set_major_formatter(date_format)

# Optionally, place a major tick every hour and minor tick every 30 minutes:
ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=30))

# 5) Labels and title with bold font
ax.set_title('Clear Sky Index (CSI) vs. Time', fontsize=18, fontweight='bold')
ax.set_xlabel('Time', fontsize=16, fontweight='bold')
ax.set_ylabel('Clear Sky Index (CSI)', fontsize=16, fontweight='bold')

# 6) Y-axis limits and tick font size
ax.set_ylim(0, 1.5)
ax.set_yticks([0.0, 0.5, 1.0, 1.5])  # explicit ticks
# (If you want minor ticks on the y-axis, you can add: ax.yaxis.set_minor_locator(...) )
ax.tick_params(axis='y', labelsize=12, width=1.5, length=6)

# 7) Grid styling: dashed major, dotted minor
ax.grid(True, which='major', linestyle='--', linewidth=0.8, alpha=0.7)
ax.grid(True, which='minor', linestyle='', linewidth=0.5, alpha=0.5)

# 8) Rotate x‐tick labels for readability
plt.xticks(rotation=45)

# 9) (Optional) If you want a legend
# legend = ax.legend(fontsize=14, frameon=True)
# legend.get_frame().set_linewidth(1.5)

# 10) Tight layout and save
plt.tight_layout()
output_path = os.path.join(data_folder_sonic, 'csi_plot_report_style.jpg')
plt.savefig(output_path, dpi=300)
plt.show()

In [ ]:
print(merged_data_csi)

In [ ]:
# Convert TIMESTAMP to datetime if it's not already
all_data_combined_albedo['TIMESTAMP'] = pd.to_datetime(all_data_combined_albedo['TIMESTAMP'], errors='coerce')
merged_data_csi['TIMESTAMP'] = pd.to_datetime(merged_data_csi['TIMESTAMP'], errors='coerce')

# 2. Set the TIMESTAMP column as the index for both DataFrames
all_data_combined_albedo.set_index('TIMESTAMP', inplace=True)
merged_data_csi.set_index('TIMESTAMP', inplace=True)

# 3. Extract only the CSI column from merged_data_csi
csi_series = merged_data_csi[['CSI']]

# 4. Merge the CSI series into all_data_combined_albedo on the timestamp index, using a left join
all_data_combined_albedo = all_data_combined_albedo.merge(
    csi_series,
    left_index=True,
    right_index=True,
    how='left'
)

# Select only numeric columns for resampling
numeric_columns = all_data_combined_albedo.select_dtypes(include='number').columns
all_data_combined_albedo_numeric = all_data_combined_albedo[numeric_columns]

# Resample the data to 10-minute intervals and calculate the mean for each interval
all_data_combined_albedo_10min = all_data_combined_albedo_numeric.resample('10T').mean()

# Calculate net radiation separately
net_radiation_10min = (
    (all_data_combined_albedo[swr_up_column] - all_data_combined_albedo[swr_column])
    .resample('10T')
    .mean() +
    (all_data_combined_albedo[lwr_up_column] - all_data_combined_albedo[lwr_down_column])
    .resample('10T')
    .mean()
)

# Combine net radiation with the resampled data
all_data_combined_albedo_10min['Net_Radiation_10min'] = net_radiation_10min

# Reset the index to make TIMESTAMP a column again
all_data_combined_albedo_10min.reset_index(inplace=True)

# Display the resulting DataFrame
print(all_data_combined_albedo_10min)


### Sonic

In [ ]:
# Directory containing Sonic data
# ⚠️ Edit this path to match your local setup before running.
data_dir = f"C:\\path\\to\\your\\Sonic\\{month_str}\\{date_str}"


# Initialize lists to store wT fluxes and corresponding timestamps
timestamps_10min = []
wT_fluxes_10min = []
wrhoqv_fluxes_10min = []
wrhoco2_fluxes_10min = []
average_temperatures_10min = []
average_temperatures_corr_10min = []
average_h2o_density_10min = []
average_co2_density_10min = []
average_wind_ux_10min = []
average_wind_uy_10min = []
average_wind_uz_10min = []
uw_flux_10min = []
vw_flux_10min = []
uv_flux_10min = []
average_wind_ux_10min_corr=[]
average_wind_uy_10min_corr=[]
uw_flux_10min_corr=[]
vw_flux_10min_corr=[]
uv_flux_10min_corr=[]

for filename in os.listdir(data_dir):
    if filename.endswith('.dat'):
        file_path = os.path.join(data_dir, filename)
        
        # Load data from .dat file, skipping the first row
        data = pd.read_csv(file_path, skiprows=1, delimiter=',', encoding='latin1', low_memory=False)
        
        # Drop any rows with missing values
        data.dropna(inplace=True)
        
        # Convert the timestamp to datetime format
        data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'], errors='coerce')
        data['Ux'] = pd.to_numeric(data['Ux'], errors='coerce')
        data['Uy'] = pd.to_numeric(data['Uy'], errors='coerce')
        data['Uz'] = pd.to_numeric(data['Uz'], errors='coerce')
        data['T_SONIC'] = pd.to_numeric(data['T_SONIC'], errors='coerce')
        data['T_SONIC_corr'] = pd.to_numeric(data['T_SONIC_corr'], errors='coerce')
        data['H2O_density'] = pd.to_numeric(data['H2O_density'], errors='coerce')
        data['CO2_density'] = pd.to_numeric(data['CO2_density'], errors='coerce')

        # Apply directional correction (rotate 116° clockwise)
        theta_deg = 116
        theta_rad = np.deg2rad(theta_deg)

        # Correct Ux and Uy (rotate into mast-aligned frame)
        #u_rot = data['Ux'] * np.cos(theta_rad) + data['Uy'] * np.sin(theta_rad)
        #v_rot = -data['Ux'] * np.sin(theta_rad) + data['Uy'] * np.cos(theta_rad)

        # Rotate Ux and Uy into corrected coordinate system
        data['Ux_corrected'] = data['Ux'] * np.cos(theta_rad) + data['Uy'] * np.sin(theta_rad)
        data['Uy_corrected'] = -data['Ux'] * np.sin(theta_rad) + data['Uy'] * np.cos(theta_rad)


# Loop through each 10-minute chunk in the file
        for i in range(0, len(data), 12000):  # 12000 samples in 10 minutes with 20 Hz frequency
            ten_minute_data = data.iloc[i:i+12000]  # Get data for 10 minutes
            
            if not ten_minute_data.empty:
                # Calculate 10-minute means
                mean_wind_ux = ten_minute_data['Ux'].mean()
                mean_wind_uy = ten_minute_data['Uy'].mean()
                mean_wind_uz = ten_minute_data['Uz'].mean()
                mean_wind_ux_corr = ten_minute_data['Ux_corrected'].mean()
                mean_wind_uy_corr = ten_minute_data['Uy_corrected'].mean()
                # Calculate the mean temperature for the 10 minutes
                mean_T_sonic = ten_minute_data['T_SONIC'].mean()
                mean_T_sonic_corr = ten_minute_data['T_SONIC_corr'].mean()
                mean_h2o_density = ten_minute_data['H2O_density'].mean()
                mean_co2_density = ten_minute_data['CO2_density'].mean()

                # Calculate perturbations for wind components
                u_prime = ten_minute_data['Ux'] - mean_wind_ux
                v_prime = ten_minute_data['Uy'] - mean_wind_uy
                w_prime = ten_minute_data['Uz'] - mean_wind_uz
                u_prime_corr = ten_minute_data['Ux_corrected'] - mean_wind_ux_corr
                v_prime_corr = ten_minute_data['Uy_corrected'] - mean_wind_uy_corr

                # Calculate momentum fluxes
                uw_flux = u_prime * w_prime
                vw_flux = v_prime * w_prime
                uv_flux = u_prime * v_prime
                uw_flux_corr = u_prime_corr * w_prime
                vw_flux_corr = v_prime_corr * w_prime
                uv_flux_corr = u_prime_corr * v_prime_corr
                
                t_sonic_prime = ten_minute_data['T_SONIC_corr'] - mean_T_sonic_corr
                
                # Calculate wT flux (w'T' flux)
                wT_flux = (w_prime * (t_sonic_prime + 273.15)).mean()
                
                # w'qv'flux
                rhoqv_prime = ten_minute_data['H2O_density'] - mean_h2o_density
                wrhoqv_flux = (w_prime * rhoqv_prime).mean()
                
                # rhoCo2'
                rhoco2_prime = ten_minute_data['CO2_density'] - mean_co2_density
                # w'rhoco2'
                wrhoco2_flux = (rhoco2_prime * w_prime).mean()
        
                # Store data in lists
                timestamps_10min.append(ten_minute_data['TIMESTAMP'].iloc[0].floor('10T'))
                wT_fluxes_10min.append(wT_flux)
                wrhoqv_fluxes_10min.append(wrhoqv_flux)
                wrhoco2_fluxes_10min.append(wrhoco2_flux)
                average_temperatures_10min.append(mean_T_sonic + 273.15)
                average_temperatures_corr_10min.append(mean_T_sonic_corr + 273.15)
                average_h2o_density_10min.append(mean_h2o_density)
                average_co2_density_10min.append(mean_co2_density)
                average_wind_ux_10min.append(mean_wind_ux)
                average_wind_uy_10min.append(mean_wind_uy)
                average_wind_uz_10min.append(mean_wind_uz)
                average_wind_ux_10min_corr.append(mean_wind_ux_corr)
                average_wind_uy_10min_corr.append(mean_wind_uy_corr)
                uw_flux_10min.append(uw_flux.mean())
                vw_flux_10min.append(vw_flux.mean())
                uv_flux_10min.append(uv_flux.mean())
                uw_flux_10min_corr.append(uw_flux_corr.mean())
                vw_flux_10min_corr.append(vw_flux_corr.mean())
                uv_flux_10min_corr.append(uv_flux_corr.mean())
# Create DataFrame for the collected data
flux_data_10min = pd.DataFrame({
    'TIMESTAMP': timestamps_10min,
    'wT_Flux': wT_fluxes_10min,
    'wrhoqv_Flux': wrhoqv_fluxes_10min,
    'Average_Temperature': average_temperatures_10min,
    'Average_Temperature_Corr': average_temperatures_corr_10min,
    'Average_H2O_Density': average_h2o_density_10min,
    'Average_CO2_Density': average_co2_density_10min,
    'wrhoCO2_Flux': wrhoco2_fluxes_10min,
    'Average_Wind_Ux': average_wind_ux_10min,
    'Average_Wind_Uy': average_wind_uy_10min,
    'Average_Wind_Uz': average_wind_uz_10min,
    'Average_Wind_Ux_corrected': average_wind_ux_10min_corr,
    'Average_Wind_Uy_corrected': average_wind_uy_10min_corr,

    'uw_flux': uw_flux_10min,
    'vw_flux': vw_flux_10min,
    'uv_flux': uv_flux_10min,
    'uw_flux_corr': uw_flux_10min_corr,
    'vw_flux_corr': vw_flux_10min_corr,
    'uv_flux_corr': uv_flux_10min_corr
})

print(flux_data_10min)

In [ ]:
print(flux_data_10min.columns)

In [ ]:
# Merge the resampled net radiation with flux_data_10min
#merged_data_10min = pd.merge_asof(flux_data_10min.sort_values('TIMESTAMP'),
 #                                 all_data_combined_albedo_10min.sort_values('TIMESTAMP'),
  #                                on='TIMESTAMP', direction='nearest')
merged_data_10min = pd.merge(
    flux_data_10min,
    all_data_combined_albedo_10min,
    on='TIMESTAMP',  # Only merge on exact timestamp matches
    how='inner'      # Use 'inner' to keep only rows with matching timestamps in both dataframes
)

# Merge dataframes based on overlapping timestamps
#merged_data_10min = pd.merge_asof(merged_data_10min.sort_values('TIMESTAMP'),
  #  all_pressure_data_10min.sort_values('TIMESTAMP'),
   # on='TIMESTAMP',
    #direction='nearest')
    
merged_data_10min = pd.merge(
    merged_data_10min,
    all_pressure_data_10min,
    on='TIMESTAMP',  # Only merge on exact timestamp matches
    how='inner'      # Use 'inner' to keep only rows with matching timestamps in both dataframes
)

#Calculations
merged_data_10min['tau_xz'] = -merged_data_10min['rho_air_Tv'] * merged_data_10min['uw_flux']
merged_data_10min['tau_yz'] = -merged_data_10min['rho_air_Tv'] * merged_data_10min['vw_flux']
merged_data_10min['tau_xy'] = -merged_data_10min['rho_air_Tv'] * merged_data_10min['uv_flux']
merged_data_10min['tau_xz_corr'] = -merged_data_10min['rho_air_Tv'] * merged_data_10min['uw_flux_corr']
merged_data_10min['tau_yz_corr'] = -merged_data_10min['rho_air_Tv'] * merged_data_10min['vw_flux_corr']
merged_data_10min['tau_xy_corr'] = -merged_data_10min['rho_air_Tv'] * merged_data_10min['uv_flux_corr']

# Calculate sensible heat flux (SHF)
merged_data_10min['SHF'] = merged_data_10min['wT_Flux'] * merged_data_10min['rho_air_Tv'] * Cp

# Calculate latent heat flux (LHF)
merged_data_10min['qv_sonic'] = merged_data_10min['Average_H2O_Density'] / merged_data_10min['rho_air_Tv']
merged_data_10min['wqv_Flux'] = merged_data_10min['wrhoqv_Flux'] / merged_data_10min['rho_air_Tv']
merged_data_10min['LHF'] = merged_data_10min['wqv_Flux'] * merged_data_10min['rho_air_Tv'] * Lv / 1000

## Calculate ground heat flux (G)
merged_data_10min['G'] = -(merged_data_10min['SHF'] + merged_data_10min['LHF'] + merged_data_10min['Net_Radiation_10min'])

# Calculate wind speed
merged_data_10min['Wind_Speed'] = np.sqrt(
    merged_data_10min['Average_Wind_Ux']**2 +
    merged_data_10min['Average_Wind_Uy']**2 +
    merged_data_10min['Average_Wind_Uz']**2
)


merged_data_10min['PPM_CO2'] = merged_data_10min['Average_CO2_Density'] * m_atm / (m_co2 * merged_data_10min['rho_air_Tv'])
merged_data_10min['F_CO2'] = merged_data_10min['wrhoCO2_Flux'] * m_atm / (m_co2 * merged_data_10min['rho_air_Tv'])


# Display the merged dataframe
print(merged_data_10min)

# Define the file path where you want to save the CSV within the data directory
output_file = os.path.join(data_dir, 'merged_data_10min.csv')

# Save merged_data_10min to CSV
merged_data_10min.to_csv(output_file, index=False)

print(f"Saved merged data to {output_file}")

In [ ]:
print(merged_data_10min.columns)

In [ ]:
# Create a figure with 2x2 subplots
fig, axs = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: SHF and LHF vs Time
axs[0, 0].scatter(merged_data_10min['TIMESTAMP'], merged_data_10min['SHF'], s=10, alpha=0.7, color='black', label='SHF')
axs[0, 0].scatter(merged_data_10min['TIMESTAMP'], merged_data_10min['LHF'], s=10, alpha=0.7, color='green',label='LHF')
axs[0, 0].xaxis.set_major_formatter(DateFormatter('%H:%M'))
axs[0, 0].set_xlabel('Time')
axs[0, 0].set_ylabel('Heat Flux (W/m^2)')
axs[0, 0].set_title('SHF, LHF vs Time (averaged in 10 min intervals)')
axs[0, 0].legend()
axs[0, 0].grid(True)

# Plot 2: G vs Time
axs[0, 1].scatter(merged_data_10min['TIMESTAMP'], merged_data_10min['G'], color='red', s=10, alpha=0.7)
axs[0, 1].xaxis.set_major_formatter(DateFormatter('%H:%M'))
axs[0, 1].set_xlabel('Time')
axs[0, 1].set_ylabel('G (W/m^2)')
axs[0, 1].set_title('Heat flux into the ground vs Time (averaged in 10 min intervals)')
axs[0, 1].grid(True)

# Plot 3: F_CO2 vs Time
axs[1, 0].scatter(merged_data_10min['TIMESTAMP'], merged_data_10min['F_CO2'], s=10, color='indigo', alpha=0.7)
axs[1, 0].xaxis.set_major_formatter(DateFormatter('%H:%M'))
axs[1, 0].set_xlabel('Time')
axs[1, 0].set_ylabel('F_CO2 (ppm*ms^-1)')
axs[1, 0].set_title('CO2 flux vs Time (averaged in 10 min intervals)')
axs[1, 0].grid(True)
axs[1, 0].set_ylim(-0.5, 0.5)

# Plot 4: Net Radiation vs Time (assuming you have this data)
axs[1, 1].scatter(merged_data_10min['TIMESTAMP'], merged_data_10min['Net_Radiation_10min'], color='blue', s=10, alpha=0.7)
axs[1, 1].xaxis.set_major_formatter(DateFormatter('%H:%M'))
axs[1, 1].set_xlabel('Time')
axs[1, 1].set_ylabel('Net Radiation (W/m^2)')
axs[1, 1].set_title('Net Radiation vs Time')
axs[1, 1].grid(True)

# Adjust layout to prevent overlap
fig.tight_layout()

# Save the figure as a PNG file in the data directory
plt.savefig(os.path.join(data_dir, 'combined_plots_10min.png'), dpi=300)

# Show the plot
plt.show()